<a href="https://colab.research.google.com/github/M-Rasit/Deep-Learning-Study-and-Projects/blob/main/Hands-On%20Machine%20Learning%20with%20Scikit-Learn%20and%20TensorFlow/Chapter_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"
import numpy as np
import matplotlib.pyplot as plt

**Tensorflow Intializers**

In [2]:
[name for name in dir(keras.initializers) if not name.startswith("_")]

['Constant',
 'GlorotNormal',
 'GlorotUniform',
 'HeNormal',
 'HeUniform',
 'Identity',
 'Initializer',
 'LecunNormal',
 'LecunUniform',
 'Ones',
 'Orthogonal',
 'RandomNormal',
 'RandomUniform',
 'TruncatedNormal',
 'VarianceScaling',
 'Zeros',
 'constant',
 'deserialize',
 'get',
 'glorot_normal',
 'glorot_uniform',
 'he_normal',
 'he_uniform',
 'identity',
 'lecun_normal',
 'lecun_uniform',
 'ones',
 'orthogonal',
 'random_normal',
 'random_uniform',
 'serialize',
 'truncated_normal',
 'variance_scaling',
 'zeros']

To use the leaky ReLU activation function, we must create a LeakyReLU instance

In [ ]:
leaky_relu = keras.layers.LeakyReLU(alpha=0.2)
layer = keras.layers.Dense(10, activation=leaky_relu,
                           kernel_initializer="he_normal")

For PReLU;

In [ ]:
prelu = keras.layers.PReLU()
layer = keras.layers.Dense(10, activation=prelu,
                           kernel_initializer="he_normal")

for SELU activation, we need to use LeCUN as kernel_initializer

In [ ]:
layer = keras.layers.Dense(10, activation="selu",
                           kernel_initializer="lecun_normal")

Training a neural network with Fashion MNIST dataset using SELU;

In [3]:
(X_train_all, y_train_all), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_all = X_train_all / 255
X_test = X_test / 255
X_valid, X_train = X_train_all[:5000], X_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

4422102/4422102 [==============================] - 0s 0us/step


In [4]:
np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"))

for layer in range(99):
  model.add(keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"))

model.add(keras.layers.Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])


We have to scale the inputs to mean 0 and standard deviation 1;

In [5]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [ ]:
history = model.fit(X_train_scaled, y_train, epochs=5, 
                    validation_data=[X_valid_scaled, y_valid])

Epoch 1/5
1719/1719 [==============================] - 44s 20ms/step - loss: 1.2531 - accuracy: 0.5341 - val_loss: 0.8964 - val_accuracy: 0.6898
Epoch 2/5
1719/1719 [==============================] - 33s 19ms/step - loss: 0.7643 - accuracy: 0.7230 - val_loss: 0.6466 - val_accuracy: 0.7710
Epoch 3/5
1719/1719 [==============================] - 35s 20ms/step - loss: 0.6431 - accuracy: 0.7649 - val_loss: 0.5967 - val_accuracy: 0.7826
Epoch 4/5
1719/1719 [==============================] - 33s 19ms/step - loss: 0.6410 - accuracy: 0.7650 - val_loss: 0.5713 - val_accuracy: 0.7936
Epoch 5/5
1719/1719 [==============================] - 33s 19ms/step - loss: 0.5403 - accuracy: 0.8067 - val_loss: 0.5084 - val_accuracy: 0.8262


**Batch Normalization**

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_104 (Dense)           (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_105 (Dense)           (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)             

In [ ]:
[(variables.name, variables.trainable) for variables in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=10, 
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 13s 7ms/step - loss: 0.8508 - accuracy: 0.7146 - val_loss: 0.5495 - val_accuracy: 0.8148
Epoch 2/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.5815 - accuracy: 0.7981 - val_loss: 0.4750 - val_accuracy: 0.8376
Epoch 3/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.5184 - accuracy: 0.8184 - val_loss: 0.4383 - val_accuracy: 0.8504
Epoch 4/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.4869 - accuracy: 0.8295 - val_loss: 0.4163 - val_accuracy: 0.8576
Epoch 5/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.4632 - accuracy: 0.8369 - val_loss: 0.3995 - val_accuracy: 0.8624
Epoch 6/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.4430 - accuracy: 0.8437 - val_loss: 0.3868 - val_accuracy: 0.8674
Epoch 7/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.4277 - accuracy: 0.8498 - val_loss: 0.3796 - val_ac

Applying Batch Normalization before the activation function. We can use "use_bias=False" because BatchNormalization layer has some bias term too.

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer=keras.optimizers.SGD(1e-3),
              metrics=["accuracy"])

In [ ]:
model.fit(X_train, y_train, epochs=10,
          validation_data=[X_valid, y_valid])

Epoch 1/10
1719/1719 [==============================] - 14s 7ms/step - loss: 1.0306 - accuracy: 0.6815 - val_loss: 0.6691 - val_accuracy: 0.7932
Epoch 2/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.6799 - accuracy: 0.7820 - val_loss: 0.5549 - val_accuracy: 0.8242
Epoch 3/10
1719/1719 [==============================] - 13s 7ms/step - loss: 0.5927 - accuracy: 0.8057 - val_loss: 0.4989 - val_accuracy: 0.8378
Epoch 4/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.5451 - accuracy: 0.8167 - val_loss: 0.4644 - val_accuracy: 0.8490
Epoch 5/10
1719/1719 [==============================] - 13s 7ms/step - loss: 0.5160 - accuracy: 0.8250 - val_loss: 0.4411 - val_accuracy: 0.8554
Epoch 6/10
1719/1719 [==============================] - 12s 7ms/step - loss: 0.4912 - accuracy: 0.8323 - val_loss: 0.4241 - val_accuracy: 0.8596
Epoch 7/10
1719/1719 [==============================] - 13s 7ms/step - loss: 0.4711 - accuracy: 0.8389 - val_loss: 0.4115 - val_ac

**Gradient Clipping**

In [ ]:
optimizer = keras.optimizer.SGD(clipvalue=1)
optimizer = keras.optimizer.SGD(clipnorm=1)

**Reusing Pretrained Layers**

In [ ]:
def split_dataset(X, y):
    y_8_or_9 = (y == 8) | (y == 9)
    y_A = y[~y_8_or_9]
    y_B = (y[y_8_or_9] == 9).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_8_or_9], y_A),
            (X[y_8_or_9], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]


In [ ]:
print("X_train_A shape:", X_train_A.shape)
print("X_train_B shape:", X_train_B.shape)

X_train_A shape: (43996, 28, 28)
X_train_B shape: (200, 28, 28)


In [ ]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for i in (300, 100, 50, 50, 50):
  model_A.add(keras.layers.Dense(i, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [ ]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [ ]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                      validation_data=[X_valid_A, y_valid_A])

Epoch 1/20
1375/1375 [==============================] - 8s 6ms/step - loss: 0.7643 - accuracy: 0.7189 - val_loss: 0.5864 - val_accuracy: 0.7975
Epoch 2/20
1375/1375 [==============================] - 9s 7ms/step - loss: 0.5598 - accuracy: 0.7956 - val_loss: 0.5102 - val_accuracy: 0.8202
Epoch 3/20
1375/1375 [==============================] - 9s 6ms/step - loss: 0.5079 - accuracy: 0.8126 - val_loss: 0.4783 - val_accuracy: 0.8267
Epoch 4/20
1375/1375 [==============================] - 8s 6ms/step - loss: 0.4780 - accuracy: 0.8217 - val_loss: 0.4571 - val_accuracy: 0.8344
Epoch 5/20
1375/1375 [==============================] - 8s 6ms/step - loss: 0.4591 - accuracy: 0.8295 - val_loss: 0.4441 - val_accuracy: 0.8394
Epoch 6/20
1375/1375 [==============================] - 7s 5ms/step - loss: 0.4445 - accuracy: 0.8337 - val_loss: 0.4357 - val_accuracy: 0.8452
Epoch 7/20
1375/1375 [==============================] - 7s 5ms/step - loss: 0.4320 - accuracy: 0.8395 - val_loss: 0.4247 - val_accuracy:

In [ ]:
model_A.save("my_model_A.h5")

In [ ]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [ ]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 1s 98ms/step - loss: 0.5702 - accuracy: 0.7050 - val_loss: 0.5124 - val_accuracy: 0.7992
Epoch 2/20
7/7 [==============================] - 0s 30ms/step - loss: 0.4642 - accuracy: 0.8500 - val_loss: 0.4359 - val_accuracy: 0.8564
Epoch 3/20
7/7 [==============================] - 0s 33ms/step - loss: 0.3888 - accuracy: 0.8950 - val_loss: 0.3806 - val_accuracy: 0.8896
Epoch 4/20
7/7 [==============================] - 0s 40ms/step - loss: 0.3337 - accuracy: 0.9350 - val_loss: 0.3249 - val_accuracy: 0.9237
Epoch 5/20
7/7 [==============================] - 0s 63ms/step - loss: 0.2887 - accuracy: 0.9400 - val_loss: 0.2942 - val_accuracy: 0.9327
Epoch 6/20
7/7 [==============================] - 0s 23ms/step - loss: 0.2569 - accuracy: 0.9450 - val_loss: 0.2595 - val_accuracy: 0.9468
Epoch 7/20
7/7 [==============================] - 0s 31ms/step - loss: 0.2269 - accuracy: 0.9600 - val_loss: 0.2324 - val_accuracy: 0.9568
Epoch 8/20
7/7 [===========

In [ ]:
model_A = keras.models.load_model("my_model_A.h5")
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1,activation="sigmoid"))

In [ ]:
for layer in model_B_on_A.layers[:-1]:
  layer.trainable=False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])

In [ ]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 0s 37ms/step - loss: 0.7301 - accuracy: 0.5550 - val_loss: 0.6924 - val_accuracy: 0.6305
Epoch 2/4
7/7 [==============================] - 0s 20ms/step - loss: 0.6934 - accuracy: 0.6000 - val_loss: 0.6561 - val_accuracy: 0.6586
Epoch 3/4
7/7 [==============================] - 0s 30ms/step - loss: 0.6588 - accuracy: 0.6300 - val_loss: 0.6273 - val_accuracy: 0.6787
Epoch 4/4
7/7 [==============================] - 0s 30ms/step - loss: 0.6309 - accuracy: 0.6450 - val_loss: 0.5994 - val_accuracy: 0.6998
Epoch 1/16
7/7 [==============================] - 1s 57ms/step - loss: 0.5363 - accuracy: 0.7350 - val_loss: 0.4271 - val_accuracy: 0.8143
Epoch 2/16
7/7 [==============================] - 0s 23ms/step - loss: 0.3886 - accuracy: 0.8650 - val_loss: 0.3210 - val_accuracy: 0.8966
Epoch 3/16
7/7 [==============================] - 0s 31ms/step - loss: 0.2949 - accuracy: 0.9400 - val_loss: 0.2665 - val_accuracy: 0.9347
Epoch 4/16
7/7 [===============

In [ ]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 6ms/step - loss: 0.0979 - accuracy: 0.9890


[0.0978974848985672, 0.9890000224113464]

In [ ]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 3ms/step - loss: 0.0838 - accuracy: 0.9825


[0.08375254273414612, 0.9825000166893005]

**Faster Optimizers**

*Momentum Optimization*

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9)

*Nesterov Accelerated Gradient*

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9, nesterov=True)

*AdaGrad* (It can be used for simpler tasks such as Linear Regression)

In [ ]:
optimizer = keras.optimizers.AdaGrad(learning_rate=1e-3)

*RMSProp*

In [ ]:
optimizer = keras.optimizers.RMSprop(learning_rate=1e-3, rho=0.9)

*Adam*

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=1e-3, beta_1=0.9, beta_2=0.999)

*Adamax*

In [ ]:
optimizer = keras.optimizers.Adamax(learning_rate=1e-3, beta_1=0.9, beta_2=0.999)

*Nadam*

In [ ]:
optimizer = keras.optimizers.Nadam(learning_rate=1e-3, beta_1=0.9, beta_2=0.999)

**Learning Rate Scheduling**

*Power Scheduling*

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=1e-4)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(10, activation="softmax"),
])

model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

n_epochs = 25
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/25
1719/1719 [==============================] - 33s 18ms/step - loss: 1.1957 - accuracy: 0.5971 - val_loss: 0.8528 - val_accuracy: 0.7100
Epoch 2/25
1719/1719 [==============================] - 36s 21ms/step - loss: 0.7849 - accuracy: 0.7296 - val_loss: 0.7170 - val_accuracy: 0.7554
Epoch 3/25
1719/1719 [==============================] - 29s 17ms/step - loss: 0.6924 - accuracy: 0.7599 - val_loss: 0.6544 - val_accuracy: 0.7728
Epoch 4/25
1719/1719 [==============================] - 33s 19ms/step - loss: 0.6419 - accuracy: 0.7773 - val_loss: 0.6156 - val_accuracy: 0.7838
Epoch 5/25
1719/1719 [==============================] - 32s 19ms/step - loss: 0.6085 - accuracy: 0.7884 - val_loss: 0.5884 - val_accuracy: 0.7918
Epoch 6/25
1719/1719 [==============================] - 31s 18ms/step - loss: 0.5840 - accuracy: 0.7972 - val_loss: 0.5679 - val_accuracy: 0.8004
Epoch 7/25
1719/1719 [==============================] - 31s 18ms/step - loss: 0.5652 - accuracy: 0.8038 - val_loss: 0.5518 -

*Exponential Scheduling*

In [ ]:
def exponential_decay(lr0, s):
  def exponential_decay_fn(epoch):
    return lr0 * 0.1**(epoch / s)
  return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0=0.01, s=20)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
n_epochs=25

lr_scheduler=keras.callbacks.LearningRateScheduler(exponential_decay_fn)
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid),
                    callbacks=[lr_scheduler])

Epoch 1/25
1719/1719 [==============================] - 12s 7ms/step - loss: 0.5245 - accuracy: 0.8260 - val_loss: 0.4476 - val_accuracy: 0.8454
Epoch 2/25
1719/1719 [==============================] - 17s 10ms/step - loss: 0.3697 - accuracy: 0.8658 - val_loss: 0.3440 - val_accuracy: 0.8792
Epoch 3/25
1719/1719 [==============================] - 13s 8ms/step - loss: 0.3291 - accuracy: 0.8806 - val_loss: 0.3585 - val_accuracy: 0.8742
Epoch 4/25
1719/1719 [==============================] - 12s 7ms/step - loss: 0.3044 - accuracy: 0.8870 - val_loss: 0.3673 - val_accuracy: 0.8782
Epoch 5/25
1719/1719 [==============================] - 11s 7ms/step - loss: 0.2797 - accuracy: 0.8980 - val_loss: 0.4119 - val_accuracy: 0.8628
Epoch 6/25
1719/1719 [==============================] - 10s 6ms/step - loss: 0.2583 - accuracy: 0.9038 - val_loss: 0.3693 - val_accuracy: 0.8836
Epoch 7/25
1719/1719 [==============================] - 11s 7ms/step - loss: 0.2450 - accuracy: 0.9099 - val_loss: 0.3754 - val_a

*Piecewise Constant Scheduling*

In [ ]:
def piecewise_constant(boundaries, values):
    boundaries = np.array([0] + boundaries)
    values = np.array(values)
    def piecewise_constant_fn(epoch):
        return values[np.argmax(boundaries > epoch) - 1]
    return piecewise_constant_fn

piecewise_constant_fn = piecewise_constant([5, 15], [0.01, 0.005, 0.001])

In [ ]:
lr_scheduler = keras.callbacks.LearningRateScheduler(piecewise_constant_fn)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 25
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid),
                    callbacks=[lr_scheduler])

Epoch 1/25
1719/1719 [==============================] - 17s 9ms/step - loss: 0.8637 - accuracy: 0.7560 - val_loss: 0.7478 - val_accuracy: 0.7828 - lr: 0.0100
Epoch 2/25
1719/1719 [==============================] - 15s 9ms/step - loss: 0.8498 - accuracy: 0.7611 - val_loss: 1.0946 - val_accuracy: 0.6738 - lr: 0.0100
Epoch 3/25
1719/1719 [==============================] - 15s 9ms/step - loss: 0.9474 - accuracy: 0.7327 - val_loss: 0.8379 - val_accuracy: 0.7396 - lr: 0.0100
Epoch 4/25
1719/1719 [==============================] - 20s 12ms/step - loss: 0.9183 - accuracy: 0.7361 - val_loss: 0.8986 - val_accuracy: 0.7600 - lr: 0.0100
Epoch 5/25
1719/1719 [==============================] - 17s 10ms/step - loss: 0.9207 - accuracy: 0.7322 - val_loss: 1.1545 - val_accuracy: 0.6422 - lr: 0.0100
Epoch 6/25
1719/1719 [==============================] - 20s 12ms/step - loss: 0.6731 - accuracy: 0.7983 - val_loss: 0.6333 - val_accuracy: 0.8178 - lr: 0.0050
Epoch 7/25
1719/1719 [===========================

*Performance Scheduling*

In [ ]:
tf.random.set_seed(1)
np.random.seed(1)

lr_scheduler = keras.optimizers.ReduceLROnPlateau(factor=0.5, patience=5)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 25
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid),
                    callbacks=[lr_scheduler])

Epoch 1/25
1719/1719 [==============================] - 17s 9ms/step - loss: 0.4729 - accuracy: 0.8356 - val_loss: 0.3962 - val_accuracy: 0.8644 - lr: 0.0010
Epoch 2/25
1719/1719 [==============================] - 16s 9ms/step - loss: 0.3430 - accuracy: 0.8740 - val_loss: 0.3285 - val_accuracy: 0.8856 - lr: 0.0010
Epoch 3/25
1719/1719 [==============================] - 14s 8ms/step - loss: 0.3070 - accuracy: 0.8880 - val_loss: 0.3350 - val_accuracy: 0.8794 - lr: 0.0010
Epoch 4/25
1719/1719 [==============================] - 15s 9ms/step - loss: 0.2785 - accuracy: 0.8980 - val_loss: 0.3272 - val_accuracy: 0.8854 - lr: 0.0010
Epoch 5/25
1719/1719 [==============================] - 14s 8ms/step - loss: 0.2584 - accuracy: 0.9027 - val_loss: 0.3533 - val_accuracy: 0.8834 - lr: 0.0010
Epoch 6/25
1719/1719 [==============================] - 16s 9ms/step - loss: 0.2412 - accuracy: 0.9099 - val_loss: 0.3601 - val_accuracy: 0.8836 - lr: 0.0010
Epoch 7/25
1719/1719 [==============================

**Avoiding Overfitting Through Regularization**

*l1 and l2 regularization*

In [ ]:
layer = keras.layer.Dense(100, activation="elu",
                          kernel_initializaer="he_normal",
                          kernel_regularizer=keras.regularizers.l2(0.01))

# l1(0.01)
# l1_l2(0.01, 0.02)

In [ ]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])

history=model.fit(X_train_scaled, y_train, epochs=2,
                  validation_data=[X_valid_scaled, y_valid])

Epoch 1/2
1719/1719 [==============================] - 7s 4ms/step - loss: 1.6359 - accuracy: 0.8118 - val_loss: 0.7524 - val_accuracy: 0.8160
Epoch 2/2
1719/1719 [==============================] - 7s 4ms/step - loss: 0.7192 - accuracy: 0.8259 - val_loss: 0.6743 - val_accuracy: 0.8502


*Dropout*

In [6]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(X_train_scaled, y_train, epochs=10,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/10
1719/1719 [==============================] - 17s 9ms/step - loss: 0.5967 - accuracy: 0.7980 - val_loss: 0.3889 - val_accuracy: 0.8620
Epoch 2/10
1719/1719 [==============================] - 15s 8ms/step - loss: 0.4282 - accuracy: 0.8439 - val_loss: 0.3482 - val_accuracy: 0.8800
Epoch 3/10
1719/1719 [==============================] - 14s 8ms/step - loss: 0.4009 - accuracy: 0.8539 - val_loss: 0.3401 - val_accuracy: 0.8728
Epoch 4/10
1719/1719 [==============================] - 14s 8ms/step - loss: 0.3821 - accuracy: 0.8597 - val_loss: 0.3419 - val_accuracy: 0.8796
Epoch 5/10
1719/1719 [==============================] - 14s 8ms/step - loss: 0.3711 - accuracy: 0.8621 - val_loss: 0.3303 - val_accuracy: 0.8736
Epoch 6/10
1719/1719 [==============================] - 14s 8ms/step - loss: 0.3611 - accuracy: 0.8664 - val_loss: 0.3101 - val_accuracy: 0.8878
Epoch 7/10
1719/1719 [==============================] - 14s 8ms/step - loss: 0.3513 - accuracy: 0.8703 - val_loss: 0.3263 - val_ac

*1.Monte-Carlo (MC) Dropout*



In [7]:
class MCDropout(keras.layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [8]:
mc_model = keras.models.Sequential([
    MCDropout(layer.rate) if isinstance(layer, keras.layers.Dropout) else layer 
    for layer in model.layers
])

In [9]:
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
mc_model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [10]:
mc_model.set_weights(model.get_weights())

In [11]:
np.round(np.mean([mc_model.predict(X_test_scaled[:1]) for sample in range(100)], axis=0), 2)

1/1 [==============================] - 0s 26ms/step


array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.07, 0.  , 0.28, 0.  , 0.65]],
      dtype=float32)